In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib as plt
from tensorflow import keras 
from tensorflow.keras.layers import Conv2D, concatenate, Dropout,MaxPool2D, MaxPooling2D, Conv2DTranspose, Activation, BatchNormalization,UpSampling2D, Add

In [ ]:
arr1 = np.random.randn(1,128,128,18)

In [ ]:
#Build the model
def conv_2d_block (x,n_filters,k_size,batchnorm = False):
#1st layer
  x= Conv2D(filters=n_filters,kernel_size=(k_size,k_size) ,padding='same', kernel_initializer = 'he_normal')(x)
  if batchnorm:
    x= BatchNormalization()(x)
  x = Activation('relu')(x)
#2nd Layer
  x= Conv2D(filters=n_filters,kernel_size=(k_size,k_size) ,padding='same', kernel_initializer = 'he_normal')(x)
  if batchnorm:
    x= BatchNormalization()(x)
  x = Activation('relu')(x)

  return x
 
def clothing_encoder(inputs,n_filters=16,conv_k_size=3,pool_size=2,batchnorm=True,dropout=.2):
  c1 = conv_2d_block(inputs,n_filters*1,conv_k_size,batchnorm)  
  p1 = MaxPool2D(pool_size=(pool_size,pool_size))(c1)
  p1 = Dropout(dropout)(p1)
  
  c2 = conv_2d_block(p1,n_filters*2,conv_k_size,batchnorm)  
  p2 = MaxPool2D(pool_size=(pool_size,pool_size))(c2)
  p2 = Dropout(dropout)(p2)

  c3 = conv_2d_block(p2,n_filters*4,conv_k_size,batchnorm)  
  p3 = MaxPool2D(pool_size=(pool_size,pool_size))(c3)
  p3 = Dropout(dropout)(p3)

  c4 = conv_2d_block(p3,n_filters*8,conv_k_size,batchnorm)  
  p4 = MaxPool2D(pool_size=(pool_size,pool_size))(c4)
  p4 = Dropout(dropout)(p4)

  c5 = conv_2d_block(p4,n_filters*16,conv_k_size,batchnorm)  
  p5 = MaxPool2D(pool_size=(pool_size,pool_size))(c5)
  p5 = Dropout(dropout)(p5)

  c6 = conv_2d_block(p5,n_filters*32,conv_k_size,batchnorm)  
  p6 = MaxPool2D(pool_size=(pool_size,pool_size))(c6)
  p6 = Dropout(dropout)(p6)

  c7 = conv_2d_block(p6,n_filters*64,conv_k_size,batchnorm)
 

  a =  tf.keras.layers.UpSampling2D(size=(8, 8), interpolation="nearest")(c7)

  c8 = conv_2d_block(a,n_filters*32,conv_k_size,batchnorm)  


  return c8


  

In [ ]:
clothing_encoder(arr1).shape

TensorShape([1, 16, 16, 512])

In [ ]:
arr2 = np.random.randn(1,128,128,3)

In [ ]:
def pose_encoder(inputs,n_filters=16,conv_k_size=3,pool_size=2,batchnorm=True,dropout=.2):
  c01 = conv_2d_block(inputs,n_filters*1,conv_k_size,batchnorm)  
  p01 = MaxPool2D(pool_size=(pool_size,pool_size))(c01)
  p01 = Dropout(dropout)(p01)
  
  c02 = conv_2d_block(p01,n_filters*2,conv_k_size,batchnorm)  
  p02 = MaxPool2D(pool_size=(pool_size,pool_size))(c02)
  p02 = Dropout(dropout)(p02)

  c03 = conv_2d_block(p02,n_filters*4,conv_k_size,batchnorm)  
  p03 = MaxPool2D(pool_size=(pool_size,pool_size))(c03)
  p03 = Dropout(dropout)(p03)

  c04 = conv_2d_block(p03,n_filters*8,conv_k_size,batchnorm)  
 

  c05 = conv_2d_block(c04,n_filters*16,conv_k_size,batchnorm)  
  

  c06 = conv_2d_block(c05,n_filters*32,conv_k_size,batchnorm)
 
  return c06


In [ ]:
pose_encoder(arr2).shape

TensorShape([1, 16, 16, 512])

In [ ]:
b = tf.keras.layers.concatenate([clothing_encoder(arr1),pose_encoder(arr2)])


In [ ]:
  b.shape


TensorShape([1, 16, 16, 1024])

In [ ]:
def res_identity(x, filters): 
  #renet block where dimension doesnot change.
  #The skip connection is just simple identity conncection
  #we will have 3 blocks and then input will be added

  x_skip = x # this will be used for addition with the residual block 
  f1, f2 = filters

  #first block 
  x = Conv2D(f1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  #second block # bottleneck (but size kept same with padding)
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same' )(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  # third block activation used after adding the input
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
  x = BatchNormalization()(x)
  # x = Activation('relu')(x)
  
  # add the input 
  x = Add()([x, x_skip])
  x = Activation('relu')(x)

  return x

In [ ]:
res_identity(b,(64,1024)).shape

TensorShape([1, 16, 16, 1024])

In [ ]:
def res_conv(x, s, filters):
  '''
  here the input size changes''' 
  x_skip = x
  f1, f2 = filters

  # first block
  x = Conv2D(f1, kernel_size=(1, 1), strides=(s, s), padding='valid')(x)
  # when s = 2 then it is like downsizing the feature map
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  # second block
  x = Conv2D(f1, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  #third block
  x = Conv2D(f2, kernel_size=(1, 1), strides=(1, 1), padding='valid')(x)
  x = BatchNormalization()(x)

  # shortcut 
  x_skip = Conv2D(f2, kernel_size=(1, 1), strides=(s, s), padding='valid')(x_skip)
  x_skip = BatchNormalization()(x_skip)

  # add 
  x = Add()([x, x_skip])
  x = Activation('relu')(x)

  return x

In [ ]:
c = res_conv(b,1,(64,512))

In [ ]:
c.shape

TensorShape([1, 16, 16, 512])

In [ ]:
#cloth decoder 
def clothing_decoder(inputs,n_filters=16,conv_k_size=3,pool_size=2,batchnorm=True,dropout=.2):
  u6 = Conv2DTranspose(filters=n_filters *16 ,kernel_size=(3,3), strides=(2,2),padding='same')(c)
  u6 = Dropout(dropout)(u6)
  c7 = conv_2d_block(u6,n_filters *16 , conv_k_size,batchnorm)

  u7 = Conv2DTranspose(filters=n_filters *8,kernel_size=(3,3), strides=(2,2),padding='same')(c7)
  u7 = Dropout(dropout)(u7)
  c8 = conv_2d_block(u7,n_filters *8 , conv_k_size,batchnorm)

  u8 = Conv2DTranspose(filters=n_filters *4,kernel_size=(3,3), strides=(2,2),padding='same')(c8)
  u8 = Dropout(dropout)(u8)
  c9 = conv_2d_block(u8,n_filters *4, conv_k_size,batchnorm)

  c10 = Conv2D(filters=18, kernel_size=(1,1),activation='softmax')(c9)
  
  return c10


In [ ]:
d =  clothing_decoder(c)

In [ ]:
d.shape

TensorShape([1, 128, 128, 18])

In [ ]:
def resnet50():

  input_im = Input(shape=(train_im.shape[1], train_im.shape[2], train_im.shape[3])) # cifar 10 images size
  x = ZeroPadding2D(padding=(3, 3))(input_im)

  # 1st stage
  # here we perform maxpooling, see the figure above

  x = BatchNormalization()(x)
  x = Activation(activations.relu)(x)


  #2nd stage 
  # frm here on only conv block and identity block, no pooling

  x = res_identity(x, filters=(64, 256))
  x = res_identity(x, filters=(64, 256))

  # 3rd stage

  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))
  x = res_identity(x, filters=(128, 512))

  # 4th stage

  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))
  x = res_identity(x, filters=(256, 1024))

  # 5th stage

  x = res_identity(x, filters=(512, 2048))
  x = res_identity(x, filters=(512, 2048))

  # ends with average pooling and dense connection

  x = AveragePooling2D((2, 2), padding='same')(x)

  x = Flatten()(x)
  x = Dense(len(class_types), activation='softmax', kernel_initializer='he_normal')(x) #multi-class

  # define the model 

  model = Model(inputs=input_im, outputs=x, name='Resnet50')

  return model
